<a href="https://colab.research.google.com/github/agarwalanant/DeepLearning/blob/master/BoltzmannMachines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!wget http://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip
!unzip Boltzmann_Machines.zip -d gdrive/My\ \Drive/DeepLearning

--2018-11-01 14:24:51--  http://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip
Resolving www.superdatascience.com (www.superdatascience.com)... 104.27.172.21, 104.27.173.21, 2606:4700:30::681b:ac15, ...
Connecting to www.superdatascience.com (www.superdatascience.com)|104.27.172.21|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip [following]
--2018-11-01 14:24:51--  https://www.superdatascience.com/wp-content/uploads/2017/03/Boltzmann_Machines.zip
Connecting to www.superdatascience.com (www.superdatascience.com)|104.27.172.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23195800 (22M) [application/zip]
Saving to: ‘Boltzmann_Machines.zip’

Boltzmann_Machines. 100%[===================>]  22.12M  5.42MB/s    in 4.2s    

2018-11-01 14:24:55 (5.31 MB/s) - ‘Boltzmann_Machines.zip’ saved [23195800/23195800]

Archi

In [3]:
!pip3 install torch torchvision
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

    100% |████████████████████████████████| 519.5MB 25kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58fda000 @  0x7fb3ff8282a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
    100% |████████████████████████████████| 61kB 20.9MB/s 
    100% |████████████████████████████████| 2.0MB 3.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
#importing database
ratings = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-1m/ratings.dat", sep ="::",header = None,engine = 'python',encoding = 'latin-1')
users = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-1m/users.dat", sep ="::",header = None,engine = 'python',encoding = 'latin-1')
movies = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-1m/movies.dat", sep ="::",header = None,engine = 'python',encoding = 'latin-1')

In [0]:
#preparing the training set and test test
training_set = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-100k/u1.base", delimiter ='\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv("gdrive/My Drive/DeepLearning/Boltzmann_Machines/ml-100k/u1.test", delimiter ='\t')
test_set = np.array(test_set, dtype = 'int')

In [0]:
#getting the number of users and movies
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))

In [0]:
#Converting data into an array with users in lines and movies in columns
#We will make list of list not a 2d numpy array as we have done before as we are using pytorch.
#1st list for 1st user 2nd list for 2nd user 
def convert(data):
    new_data = []
    for id_users in range(1,nb_users+1):
      id_movies =  data[:,1][data[:,0] == id_users]
      id_ratings =  data[:,2][data[:,0] == id_users]
      ratings = np.zeros(nb_movies)
      ratings[id_movies-1] = id_ratings
      new_data.append(list(ratings))
    return new_data

    

In [0]:
training_set = convert(training_set)
test_set = convert(test_set)

In [0]:
#Converting data into pytorch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


**Restricted Boltzman Machine**

In [0]:
#Converting the ratings to binary ratings 1 liked 0 not liked 
training_set[training_set == 0] = -1
training_set[training_set == 1 ] = 0
training_set[training_set == 2 ] = 0
training_set[training_set >= 3 ] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1 ] = 0
test_set[test_set == 2 ] = 0
test_set[test_set >= 3 ] = 1


In [0]:
#Architecture of Boltzman Machine 
class RBM():
  def __init__(self, nv, nh):
    self.W